In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random
NB_CLASS=59
SEED=888
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
torch.backends.cudnn.benchmark = True

In [2]:
BATCH_SIZE=32
IMAGE_SIZE=299    # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    

In [3]:
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.inceptionv4_finetune(NB_CLASS)
    model.cuda()
    print('Done')
    return model
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/InceptionV4/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.9
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3 or epoch==4 or epoch==5:
            lr=0.00006
            momentum=0.95
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if epoch==6:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))        
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/InceptionV4/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
           # optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            #optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
             #optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'InceptionV4', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'InceptionV4', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
train(60)

[+] loading model... Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.101042,loss:3.930136
[epoch:0,batch:59]:acc: 0.210417,loss:3.641856
[epoch:0,batch:89]:acc: 0.283333,loss:3.404907
[epoch:0,batch:119]:acc: 0.324219,loss:3.216955
[epoch:0,batch:149]:acc: 0.354583,loss:3.060818
[epoch:0,batch:179]:acc: 0.383333,loss:2.923591
[epoch:0,batch:209]:acc: 0.407292,loss:2.810456
[epoch:0,batch:239]:acc: 0.422135,loss:2.709611
[epoch:0,batch:269]:acc: 0.437384,loss:2.612714
[epoch:0,batch:299]:acc: 0.449271,loss:2.538002
[epoch:0,batch:299]: val_loss:1.775880,val_acc:0.581406,val_total:4539
[epoch:0,batch:329]:acc: 0.462595,loss:2.461736
[epoch:0,batch:359]:acc: 0.472656,loss:2.395933
[epoch:0,batch:389]:acc: 0.484215,loss:2.337884
[epoch:0,batch:419]:acc: 0.492336,loss:2.285381
[epoch:0,batch:449]:acc: 0.499583,loss:2.237738
[epoch:0,batch:479]:acc: 0.505924,loss:2.191929
[epoch:0,batch:509]:acc: 0.512929,loss:2.153266
[epoch:0,batch:539]:acc: 0.519965,loss:2.11183

[epoch:4,batch:149]:acc: 0.740833,loss:0.833128
[epoch:4,batch:179]:acc: 0.740451,loss:0.830079
[epoch:4,batch:209]:acc: 0.740774,loss:0.824087
[epoch:4,batch:239]:acc: 0.736328,loss:0.835119
[epoch:4,batch:269]:acc: 0.738426,loss:0.829073
[epoch:4,batch:299]:acc: 0.739375,loss:0.826022
[epoch:4,batch:299]: val_loss:0.878065,val_acc:0.735184,val_total:4539
[epoch:4,batch:329]:acc: 0.737311,loss:0.826897
[epoch:4,batch:359]:acc: 0.737413,loss:0.828679
[epoch:4,batch:389]:acc: 0.735737,loss:0.827704
[epoch:4,batch:419]:acc: 0.736384,loss:0.829635
[epoch:4,batch:449]:acc: 0.739236,loss:0.823240
[epoch:4,batch:479]:acc: 0.739453,loss:0.821778
[epoch:4,batch:509]:acc: 0.739583,loss:0.819113
[epoch:4,batch:539]:acc: 0.739988,loss:0.817136
[epoch:4,batch:569]:acc: 0.740844,loss:0.816666
[epoch:4,batch:599]:acc: 0.739896,loss:0.816102
[epoch:4,batch:599]: val_loss:0.870882,val_acc:0.740251,val_total:4539
[epoch:4,batch:629]:acc: 0.741171,loss:0.813789
[epoch:4,batch:659]:acc: 0.740767,loss:0.8

[epoch:8,batch:299]: val_loss:0.954132,val_acc:0.723287,val_total:4539
[epoch:8,batch:329]:acc: 0.746307,loss:0.788703
[epoch:8,batch:359]:acc: 0.746267,loss:0.791374
[epoch:8,batch:389]:acc: 0.746955,loss:0.793379
[epoch:8,batch:419]:acc: 0.744866,loss:0.798215
[epoch:8,batch:449]:acc: 0.743681,loss:0.800307
[epoch:8,batch:479]:acc: 0.743424,loss:0.798570
[epoch:8,batch:509]:acc: 0.743627,loss:0.798411
[epoch:8,batch:539]:acc: 0.742477,loss:0.800734
[epoch:8,batch:569]:acc: 0.742379,loss:0.801829
[epoch:8,batch:599]:acc: 0.742031,loss:0.803142
[epoch:8,batch:599]: val_loss:0.903363,val_acc:0.738268,val_total:4539
[epoch:8,batch:629]:acc: 0.741766,loss:0.806969
[epoch:8,batch:659]:acc: 0.741714,loss:0.807631
[epoch:8,batch:689]:acc: 0.741078,loss:0.809138
[epoch:8,batch:719]:acc: 0.741276,loss:0.806826
[epoch:8,batch:749]:acc: 0.742083,loss:0.805437
[epoch:8,batch:779]:acc: 0.742548,loss:0.805047
[epoch:8,batch:809]:acc: 0.743210,loss:0.803344
[epoch:8,batch:839]:acc: 0.743564,loss:0.8

[epoch:12,batch:479]:acc: 0.747461,loss:0.792407
[epoch:12,batch:509]:acc: 0.748284,loss:0.790578
[epoch:12,batch:539]:acc: 0.749479,loss:0.790896
[epoch:12,batch:569]:acc: 0.750713,loss:0.787650
[epoch:12,batch:599]:acc: 0.751250,loss:0.787465
[epoch:12,batch:599]: val_loss:0.878805,val_acc:0.735845,val_total:4539
[epoch:12,batch:629]:acc: 0.749454,loss:0.791399
[epoch:12,batch:659]:acc: 0.749858,loss:0.789533
[epoch:12,batch:689]:acc: 0.749864,loss:0.789864
[epoch:12,batch:719]:acc: 0.748785,loss:0.791562
[epoch:12,batch:749]:acc: 0.749500,loss:0.791106
[epoch:12,batch:779]:acc: 0.749199,loss:0.790620
[epoch:12,batch:809]:acc: 0.749151,loss:0.790228
[epoch:12,batch:839]:acc: 0.748958,loss:0.790900
[epoch:12,batch:869]:acc: 0.749389,loss:0.790240
[epoch:12,batch:899]:acc: 0.749514,loss:0.790130
[epoch:12,batch:899]: val_loss:0.876832,val_acc:0.737828,val_total:4539
[epoch:12,batch:929]:acc: 0.749933,loss:0.789716
[epoch:12,batch:959]:acc: 0.750033,loss:0.789248
[epoch:12,batch:989]:ac

[epoch:16,batch:599]: val_loss:0.955097,val_acc:0.724829,val_total:4539
[epoch:16,batch:629]:acc: 0.745337,loss:0.803995
[epoch:16,batch:659]:acc: 0.745881,loss:0.803477
[epoch:16,batch:689]:acc: 0.745924,loss:0.803867
[epoch:16,batch:719]:acc: 0.746745,loss:0.802505
[epoch:16,batch:749]:acc: 0.746542,loss:0.800804
[epoch:16,batch:779]:acc: 0.747035,loss:0.800515
[epoch:16,batch:809]:acc: 0.746528,loss:0.801535
[epoch:16,batch:839]:acc: 0.746726,loss:0.800819
[epoch:16,batch:869]:acc: 0.746013,loss:0.802461
[epoch:16,batch:899]:acc: 0.745694,loss:0.802263
[epoch:16,batch:899]: val_loss:0.865509,val_acc:0.736286,val_total:4539
[epoch:16,batch:929]:acc: 0.745363,loss:0.802766
[epoch:16,batch:959]:acc: 0.745703,loss:0.802053
[epoch:16,batch:989]:acc: 0.745139,loss:0.802791
[epoch:16] :acc: 0.745184,loss:0.803176,lr:0.000000,patience:1
[epoch:16]: val_loss:0.864583,val_acc:0.742675,
save new model loss,now loss is  0.8645831942558289
Epoch 17/59
----------
[epoch:17,batch:29]:acc: 0.756250

[epoch:20,batch:749]:acc: 0.747667,loss:0.795049
[epoch:20,batch:779]:acc: 0.748638,loss:0.794365
[epoch:20,batch:809]:acc: 0.748187,loss:0.794510
[epoch:20,batch:839]:acc: 0.747991,loss:0.795719
[epoch:20,batch:869]:acc: 0.747414,loss:0.795251
[epoch:20,batch:899]:acc: 0.747465,loss:0.795376
[epoch:20,batch:899]: val_loss:0.883469,val_acc:0.737828,val_total:4539
[epoch:20,batch:929]:acc: 0.747144,loss:0.795715
[epoch:20,batch:959]:acc: 0.747461,loss:0.796715
[epoch:20,batch:989]:acc: 0.747348,loss:0.797186
[epoch:20] :acc: 0.747391,loss:0.797488,lr:0.000000,patience:1
[epoch:20]: val_loss:0.903589,val_acc:0.739811,
Epoch 21/59
----------
loss has increased lr divide 10 lr now is :0.000000
[epoch:21,batch:29]:acc: 0.754167,loss:0.756956
[epoch:21,batch:59]:acc: 0.752604,loss:0.786661
[epoch:21,batch:89]:acc: 0.749306,loss:0.799789
[epoch:21,batch:119]:acc: 0.744010,loss:0.807095
[epoch:21,batch:149]:acc: 0.743542,loss:0.804863
[epoch:21,batch:179]:acc: 0.742535,loss:0.797395
[epoch:21,

[epoch:24,batch:899]:acc: 0.743854,loss:0.811573
[epoch:24,batch:899]: val_loss:0.979808,val_acc:0.722406,val_total:4539
[epoch:24,batch:929]:acc: 0.744825,loss:0.809391
[epoch:24,batch:959]:acc: 0.745508,loss:0.807292
[epoch:24,batch:989]:acc: 0.745770,loss:0.805501
[epoch:24] :acc: 0.745720,loss:0.805511,lr:0.000000,patience:1
[epoch:24]: val_loss:0.911978,val_acc:0.732320,
Epoch 25/59
----------
loss has increased lr divide 10 lr now is :0.000000
[epoch:25,batch:29]:acc: 0.763542,loss:0.818334
[epoch:25,batch:59]:acc: 0.756250,loss:0.797482
[epoch:25,batch:89]:acc: 0.757986,loss:0.788346
[epoch:25,batch:119]:acc: 0.751042,loss:0.794832
[epoch:25,batch:149]:acc: 0.752083,loss:0.794296
[epoch:25,batch:179]:acc: 0.753993,loss:0.790211
[epoch:25,batch:209]:acc: 0.750149,loss:0.796113
[epoch:25,batch:239]:acc: 0.752734,loss:0.791044
[epoch:25,batch:269]:acc: 0.755903,loss:0.785867
[epoch:25,batch:299]:acc: 0.756042,loss:0.781655
[epoch:25,batch:299]: val_loss:0.873247,val_acc:0.738268,va

[epoch:28] :acc: 0.745121,loss:0.798027,lr:0.000000,patience:1
[epoch:28]: val_loss:0.945407,val_acc:0.731439,
Epoch 29/59
----------
loss has increased lr divide 10 lr now is :0.000000
[epoch:29,batch:29]:acc: 0.741667,loss:0.795646
[epoch:29,batch:59]:acc: 0.755729,loss:0.784835
[epoch:29,batch:89]:acc: 0.750694,loss:0.800290
[epoch:29,batch:119]:acc: 0.745313,loss:0.803258
[epoch:29,batch:149]:acc: 0.744583,loss:0.807013
[epoch:29,batch:179]:acc: 0.746007,loss:0.801671
[epoch:29,batch:209]:acc: 0.744048,loss:0.803684
[epoch:29,batch:239]:acc: 0.742318,loss:0.801273
[epoch:29,batch:269]:acc: 0.743866,loss:0.802330
[epoch:29,batch:299]:acc: 0.743646,loss:0.802985
[epoch:29,batch:299]: val_loss:0.849386,val_acc:0.745759,val_total:4539
[epoch:29,batch:329]:acc: 0.745739,loss:0.797906
[epoch:29,batch:359]:acc: 0.743490,loss:0.800988
[epoch:29,batch:389]:acc: 0.743990,loss:0.802920
[epoch:29,batch:419]:acc: 0.743973,loss:0.798545
[epoch:29,batch:449]:acc: 0.743403,loss:0.798681
[epoch:29,

In [4]:
def reuseTrain(path,epochNum):
    writer=SummaryWriter('log/'+date+'/InceptionV4/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))   
    min_acc=max(modelParams['val_correct'],0.81)
    print('val_correct is %f'%(min_acc))
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/InceptionV4/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
        optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum,weight_decay=1e-4)
        print('lr now is %f'%(lr))
        print('now patience is %d '%(patience))
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))      
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'InceptionV4', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'InceptionV4', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
reuseTrain('../model/InceptionV4/2018-11-01_acc_best.pth',60)

[+] loading model... Done
min_loss is :0.541180
val_correct is 0.814937
Epoch 23/59
----------
lr now is 0.000100
now patience is 0 
[epoch:23,batch:29]:acc: 0.927083,loss:0.220210
[epoch:23,batch:59]:acc: 0.921875,loss:0.205425
[epoch:23,batch:89]:acc: 0.920833,loss:0.210347
[epoch:23,batch:119]:acc: 0.921615,loss:0.212755
[epoch:23,batch:149]:acc: 0.922500,loss:0.209653
[epoch:23,batch:179]:acc: 0.922743,loss:0.209175
[epoch:23,batch:209]:acc: 0.923512,loss:0.206459
[epoch:23,batch:239]:acc: 0.924479,loss:0.204577
[epoch:23,batch:269]:acc: 0.925347,loss:0.202524
[epoch:23,batch:299]:acc: 0.927396,loss:0.200184
[epoch:23,batch:299]: val_loss:0.466941,val_acc:0.826173,val_total:4539
[epoch:23,batch:329]:acc: 0.929072,loss:0.197999
[epoch:23,batch:359]:acc: 0.929167,loss:0.197435
[epoch:23,batch:389]:acc: 0.930128,loss:0.195921
[epoch:23,batch:419]:acc: 0.930655,loss:0.195363
[epoch:23,batch:449]:acc: 0.932361,loss:0.192986
[epoch:23,batch:479]:acc: 0.932943,loss:0.191181


In [5]:
def TrainWithRawData(path,epochNum):
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/InceptionV4/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'InceptionV4', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        patience=0
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'InceptionV4', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'InceptionV4', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'InceptionV4', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/InceptionV4/2018-11-01_acc_best.pth',60)

[+] loading model... Done
min_loss is :0.541180
val_correct is 0.814937
Epoch 23/59
----------
begin lr is  0.0001
[epoch:23,batch:29]:acc: 0.866667,loss:0.343528
[epoch:23,batch:59]:acc: 0.875000,loss:0.314108
[epoch:23,batch:89]:acc: 0.873264,loss:0.315965
[epoch:23,batch:119]:acc: 0.874219,loss:0.314024
[epoch:23,batch:149]:acc: 0.874583,loss:0.311133
[epoch:23,batch:179]:acc: 0.873785,loss:0.310046
[epoch:23,batch:209]:acc: 0.874405,loss:0.306651
[epoch:23,batch:239]:acc: 0.875391,loss:0.302885
[epoch:23,batch:269]:acc: 0.875463,loss:0.303127
[epoch:23,batch:299]:acc: 0.877083,loss:0.299272
[epoch:23,batch:299]: val_loss:0.398825,val_acc:0.844900,val_total:4539
save new model loss,now loss is  0.3988248109817505
save new model acc,now acc is  tensor(0.8449, device='cuda:0')
[epoch:23,batch:329]:acc: 0.876420,loss:0.297243
[epoch:23,batch:359]:acc: 0.877691,loss:0.296511
[epoch:23,batch:389]:acc: 0.879567,loss:0.293852
[epoch:23,batch:419]:acc: 0.880580,loss:0.292931
[epoch:23,batch

[epoch:26,batch:899]:acc: 0.911111,loss:0.222622
[epoch:26,batch:899]: val_loss:0.387109,val_acc:0.852170,val_total:4539
[epoch:26,batch:929]:acc: 0.911156,loss:0.222515
[epoch:26,batch:959]:acc: 0.911068,loss:0.222613
[epoch:26,batch:989]:acc: 0.911332,loss:0.222384
[epoch:26] :acc: 0.911278,loss:0.224945,lr:0.000100,patience:0
[epoch:26]: val_loss:0.423382,val_acc:0.842697,
Epoch 27/59
----------
[epoch:27,batch:29]:acc: 0.936458,loss:0.176843
[epoch:27,batch:59]:acc: 0.925521,loss:0.193973
[epoch:27,batch:89]:acc: 0.919444,loss:0.200614
[epoch:27,batch:119]:acc: 0.920052,loss:0.201676
[epoch:27,batch:149]:acc: 0.916875,loss:0.207845
[epoch:27,batch:179]:acc: 0.917535,loss:0.207062
[epoch:27,batch:209]:acc: 0.918452,loss:0.205606
[epoch:27,batch:239]:acc: 0.917057,loss:0.207392
[epoch:27,batch:269]:acc: 0.917014,loss:0.207489
[epoch:27,batch:299]:acc: 0.917292,loss:0.207714
[epoch:27,batch:299]: val_loss:0.385085,val_acc:0.855475,val_total:4539
[epoch:27,batch:329]:acc: 0.917898,loss

[epoch:30] :acc: 0.913800,loss:0.216876,lr:0.000020,patience:1
[epoch:30]: val_loss:0.463799,val_acc:0.837189,
Epoch 31/59
----------
[epoch:31,batch:29]:acc: 0.919792,loss:0.209406
[epoch:31,batch:59]:acc: 0.918229,loss:0.208172
[epoch:31,batch:89]:acc: 0.920486,loss:0.203706
[epoch:31,batch:119]:acc: 0.921875,loss:0.199520
[epoch:31,batch:149]:acc: 0.917708,loss:0.205363
[epoch:31,batch:179]:acc: 0.918056,loss:0.205786
[epoch:31,batch:209]:acc: 0.915625,loss:0.208784
[epoch:31,batch:239]:acc: 0.916797,loss:0.207726
[epoch:31,batch:269]:acc: 0.919560,loss:0.203543
[epoch:31,batch:299]:acc: 0.919896,loss:0.203214
[epoch:31,batch:299]: val_loss:0.391023,val_acc:0.852170,val_total:4539
[epoch:31,batch:329]:acc: 0.918750,loss:0.206644
[epoch:31,batch:359]:acc: 0.918316,loss:0.207482
[epoch:31,batch:389]:acc: 0.918429,loss:0.208695
[epoch:31,batch:419]:acc: 0.917857,loss:0.209926
[epoch:31,batch:449]:acc: 0.917639,loss:0.210807
[epoch:31,batch:479]:acc: 0.918099,loss:0.210252
[epoch:31,bat

[epoch:35,batch:149]:acc: 0.910625,loss:0.226631
[epoch:35,batch:179]:acc: 0.908507,loss:0.228585
[epoch:35,batch:209]:acc: 0.909524,loss:0.225421
[epoch:35,batch:239]:acc: 0.911068,loss:0.223978
[epoch:35,batch:269]:acc: 0.911343,loss:0.221882
[epoch:35,batch:299]:acc: 0.913958,loss:0.220089
[epoch:35,batch:299]: val_loss:0.385963,val_acc:0.853933,val_total:4539
[epoch:35,batch:329]:acc: 0.912500,loss:0.222044
[epoch:35,batch:359]:acc: 0.913542,loss:0.219778
[epoch:35,batch:389]:acc: 0.912420,loss:0.221292
[epoch:35,batch:419]:acc: 0.911235,loss:0.222126
[epoch:35,batch:449]:acc: 0.911458,loss:0.221843
[epoch:35,batch:479]:acc: 0.910937,loss:0.222381
[epoch:35,batch:509]:acc: 0.910784,loss:0.222030
[epoch:35,batch:539]:acc: 0.911632,loss:0.221052
[epoch:35,batch:569]:acc: 0.911842,loss:0.219759
[epoch:35,batch:599]:acc: 0.912292,loss:0.219169
[epoch:35,batch:599]: val_loss:0.389872,val_acc:0.852170,val_total:4539
[epoch:35,batch:629]:acc: 0.911458,loss:0.220911
[epoch:35,batch:659]:ac

[epoch:39,batch:299]: val_loss:0.386946,val_acc:0.855254,val_total:4539
[epoch:39,batch:329]:acc: 0.912879,loss:0.216360
[epoch:39,batch:359]:acc: 0.913802,loss:0.214906
[epoch:39,batch:389]:acc: 0.913221,loss:0.217025
[epoch:39,batch:419]:acc: 0.913244,loss:0.217232
[epoch:39,batch:449]:acc: 0.913542,loss:0.217367
[epoch:39,batch:479]:acc: 0.913477,loss:0.216991
[epoch:39,batch:509]:acc: 0.913419,loss:0.217404
[epoch:39,batch:539]:acc: 0.912847,loss:0.217664
[epoch:39,batch:569]:acc: 0.912664,loss:0.217212
[epoch:39,batch:599]:acc: 0.912813,loss:0.217046
[epoch:39,batch:599]: val_loss:0.387742,val_acc:0.852390,val_total:4539
[epoch:39,batch:629]:acc: 0.912996,loss:0.216534
[epoch:39,batch:659]:acc: 0.912737,loss:0.216682
[epoch:39,batch:689]:acc: 0.912455,loss:0.216867
[epoch:39,batch:719]:acc: 0.911936,loss:0.217466
[epoch:39,batch:749]:acc: 0.912125,loss:0.216917
[epoch:39,batch:779]:acc: 0.912059,loss:0.216649
[epoch:39,batch:809]:acc: 0.911921,loss:0.217432
[epoch:39,batch:839]:ac

In [5]:
modelParams=torch.load('../model/InceptionV4/2018-11-01_loss_best.pth')
min_loss=modelParams['val_loss']
print('min_loss is :%f'%(min_loss))
print('val_correct is %f'%(modelParams['val_correct']))

min_loss is :0.381557
val_correct is 0.856576
